# ANN models load and use

This notebook exemplifies how to load the trained ANN models and how to use them. These models use the `python_helpers` library provided in this repo.
These functions were tested using the following packages:
- `jax==0.4.4`
- `flax==0.6.6`
- `numpy==1.24.2`


In [1]:
import os
import sys
import numpy as np
import pandas as pd

from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

config.update("jax_enable_x64", True)
type_np = np.float64
type_jax = jnp.float64

sys.path.append("../")
from python_helpers import helper_get_alpha

from python_helpers.feanneos import HelmholtzModel, HelmholtzModel_Tlinear
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs

from python_helpers.transport_properties import TransportModel_PVT_Tinv, TransportModelResidual_PVT_Tinv, TransportModel_entropy
from python_helpers.transport_properties import density_diffusivity_mie6_dilute, viscosity_mie6_dilute, thermal_conductivity_mie6_dilute
from python_helpers.transport_properties import diffusivity_scaling, viscosity_scaling, thermal_conductivity_scaling
from python_helpers import linear_activation

## Loading and using FE-ANN and FE-ANN(s) EoSs

The FE-ANN and FE-ANN(s) EoS use the provided `HelmholtzModel` function. This function is required to set up the ANN of these EoS. 
The procedure to load any of the trained models is the following:
1. Load the checkpoint using the `checkpoints.restore_checkpoint` function.
1. Read the architecture of the trained model from the `'features'` attribute.
1. Create the Helmholtz model using the loaded features.
1. Load the trained parameters from the `'params'` attribute of the checkpoint


In [2]:
# Loading FE-ANN EoS
main_ckpt_folders = 'feann_eos'
name_type = 'models_Tinv_factor0.05_seed1'
ckpt_folder = os.path.join(main_ckpt_folders, name_type)
prefix_params = 'FE-ANN-EoS-params_'
# reading ANN parameters
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder, target=None, prefix=prefix_params)
# Creating Helholmoltz model
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
# Compile necessary functions for phase equilibria and thermodynamic properties computation
fun_dic_feanneos = helper_jitted_funs(helmholtz_model, helmholtz_params)

# Loading FE-ANN(s) EoS
main_ckpt_folders = 'feanns_eos'
name_type = 'models_Tinv_factor0.01_seed17'
ckpt_folder = os.path.join(main_ckpt_folders, name_type)
prefix_params = 'FE-ANN-EoS-params_'
# reading ANN parameters
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder, target=None, prefix=prefix_params)
# Creating Helholmoltz model
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
# Compile necessary functions for phase equilibria and thermodynamic properties computation
fun_dic_feannseos = helper_jitted_funs(helmholtz_model, helmholtz_params)

The `HelmholtzModel` object relies on `flax`. The EoS can be used as a flax ANN, which requires you to supply the parameters manually. For convenience, the `helper_jitted_funs` compiles the parameters into the model and provides useful functions to compute phase equilibria and thermophysical properties. These functions use `(alpha, rho, T)` as inputs. 
In this notebook, `fun_dic_feanneos` refers to compiled functions of the FE-ANN EoS, and `fun_dic_feannseos` refers to compiled functions of the FE-ANN(s) EoS.

The list of available functions is shown below.

In [3]:
print(fun_dic_feannseos.keys())

dict_keys(['helmholtz_fun', 'dhelmholtz_drho_fun', 'd2helmholtz_drho2_dT_fun', 'd2helmholtz_drho2_fun', 'd2helmholtz_fun', 'pressure_fun', 'dpressure_drho_fun', 'dpressure_drho_aux_fun', 'd2pressure_drho2_fun', 'pressure_and_chempot_fun', 'chemical_potential_residual_fun', 'entropy_residual_fun', 'internal_energy_residual_fun', 'enthalpy_residual_fun', 'gibbs_residual_fun', 'cv_residual_fun', 'cp_residual_fun', 'thermal_expansion_coeff_fun', 'thermal_pressure_coeff_fun', 'isothermal_compressibility_fun', 'joule_thomson_fun', 'thermophysical_properties_fun', 'second_virial_coefficient_fun', 'third_virial_coefficient_fun', 'virial_coefficients_fun'])


In [4]:
print(fun_dic_feannseos.keys())

dict_keys(['helmholtz_fun', 'dhelmholtz_drho_fun', 'd2helmholtz_drho2_dT_fun', 'd2helmholtz_drho2_fun', 'd2helmholtz_fun', 'pressure_fun', 'dpressure_drho_fun', 'dpressure_drho_aux_fun', 'd2pressure_drho2_fun', 'pressure_and_chempot_fun', 'chemical_potential_residual_fun', 'entropy_residual_fun', 'internal_energy_residual_fun', 'enthalpy_residual_fun', 'gibbs_residual_fun', 'cv_residual_fun', 'cp_residual_fun', 'thermal_expansion_coeff_fun', 'thermal_pressure_coeff_fun', 'isothermal_compressibility_fun', 'joule_thomson_fun', 'thermophysical_properties_fun', 'second_virial_coefficient_fun', 'third_virial_coefficient_fun', 'virial_coefficients_fun'])


Here is an example of how to use these functions.

In [5]:
# computing the alpha vdw value of the LJ particle
lambda_r = 12
lambda_a = 6 
alpha_vdw = helper_get_alpha(lambda_r, lambda_a)

# defining the state points
rho = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
T = 2. * np.ones_like(rho)
alpha = alpha_vdw * np.ones_like(rho)

# evaluating the FE-ANN EoS
print('Pressure FE-ANN EoS: ',fun_dic_feanneos['pressure_fun'](alpha, rho, T))

# evaluating the FE-ANN(s) EoS
print('Pressure FE-ANN(s) EoS: ',fun_dic_feannseos['pressure_fun'](alpha, rho, T))


Pressure FE-ANN EoS:  [0.17840788 0.33422359 0.49710743 0.72013464 1.10269866]
Pressure FE-ANN(s) EoS:  [0.17777895 0.33012058 0.49323471 0.7207129  1.10336088]


# Loading and using Transport Property models

The transport property models rely on the `TransportModel_PVT_Tinv`, `TransportModelResidual_PVT_Tinv` and `TransportModel_entropy` functions. The functions vary on the inputs/outputs.
- `TransportModel_PVT_Tinv` uses `(alpha, rho, T)` as inputs and directly returns the transport property (in the chosen scale)
- `TransportModelResidual_PVT_Tinv` use `(alpha, rho, T)` as inputs and directly returns the "residual" transport property (in the chosen scale). The dilute gas contribution can be analytically considered using the `density_diffusivity_mie6_dilute`, `viscosity_mie6_dilute`, or `thermal_conductivity_mie6_dilute` functions.
- `TransportModel_entropy` uses `(alpha, Sres)` as inputs and directly returns the reduced transport property. The transport property can be converted from a reduced scale to a normal scale using the `diffusivity_scaling`, `viscosity_scaling`, or `thermal_conductivity_scaling` functions.


In [6]:
# This is needed for the output layers
activation_dicts = {'linear': linear_activation, 'softplus': nn.softplus}

In [7]:
########################### 
# Self-diffusivity models #
###########################

folder_diff = 'selfdiff_models'
hidden_layers = 2
neurons = 30

# rhodiff model
prefix = 'rhodiff-rho-Tinv-penalty'
seed = 1
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_diff, target=None, prefix=params_prefix)
params_rhodiff = {'params': state_restored['params']}
rhodiff_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# Residual rhodiff model
prefix = 'rhodiff-rho-Tinv-residual-penalty'
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_diff, target=None, prefix=params_prefix)
params_rhodiff_res = {'params': state_restored['params']}
rhodiff_res_model = TransportModelResidual_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# Diff entropy scaling model
prefix = 'diff-entropy-penalty'
seed = 1
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_diff, target=None, prefix=params_prefix)
params_diff_entropy = {'params': state_restored['params']}
diff_entropy_model = TransportModel_entropy(features=features, output_activation=activation_dicts[activation])

################################
# Testing the different models #
################################

lambda_r = 12
lambda_a = 6 
alpha_vdw = helper_get_alpha(lambda_r, lambda_a)

# defining the state points
rho = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
T = 2. * np.ones_like(rho)
alpha = alpha_vdw * np.ones_like(rho)
Sres = fun_dic_feanneos['entropy_residual_fun'](alpha, rho, T)

# rhodiff model
diff_result = rhodiff_model.apply(params_rhodiff, alpha, rho, T) / rho

# rhodiff residual model
rhodiff_res_result = rhodiff_res_model.apply(params_rhodiff_res, lambda_r*np.ones_like(rho), rho, T)
rhodiff_dilute = density_diffusivity_mie6_dilute(lambda_r, T)
diff_res_result = (rhodiff_res_result + rhodiff_dilute) / rho

# diff entropy model
diff_entropy_result = diff_entropy_model.apply(params_diff_entropy, alpha, Sres)
diff_entropy_result = diffusivity_scaling(rho, T, diff_entropy_result, unscale=True)
# print('Self-diffusivity residual model: ', rhodiff_res_model.apply(params_rhodiff_res, alpha, rho, T) + )

print('Self-diffusivity model: ', diff_result)
print('Self-diffusivity residual model: ', diff_res_result)
print('Diff entropy model: ', diff_entropy_result)

Self-diffusivity model:  [2.70960234 1.32164389 0.84563864 0.59449846 0.4310642 ]
Self-diffusivity residual model:  [2.71416569 1.31620741 0.84071757 0.5915829  0.43041715]
Diff entropy model:  [2.90873235 1.33940173 0.83685504 0.58941387 0.41000833]


2025-03-14 15:22:35.749195: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


In [8]:
########################## 
# Shear viscosity models #
##########################

folder_visc = 'visc_models'
hidden_layers = 2
neurons = 30

# Residual logvisc model
prefix = 'logvisc-rho-Tinv-residual-penalty'
seed = 42
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_visc, target=None, prefix=params_prefix)
params_logvisc_res = {'params': state_restored['params']}
logvisc_res_model = TransportModelResidual_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# Residual logvisc model
prefix = 'logvisc-rho-Tinv-penalty'
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_visc, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# visc entropy scaling
prefix = 'visc-entropy-penalty'
seed = 0
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_visc, target=None, prefix=params_prefix)
params_visc_entropy = {'params': state_restored['params']}
visc_entropy_model = TransportModel_entropy(features=features, output_activation=activation_dicts[activation])


################################
# Testing the different models #
################################

lambda_r = 12
lambda_a = 6 
alpha_vdw = helper_get_alpha(lambda_r, lambda_a)

# defining the state points
rho = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
T = 2. * np.ones_like(rho)
alpha = alpha_vdw * np.ones_like(rho)
Sres = fun_dic_feanneos['entropy_residual_fun'](alpha, rho, T)

# visc model
visc_result = np.exp(logvisc_model.apply(params_logvisc, alpha, rho, T))

# visc residual model
visc_res_result = np.exp(logvisc_res_model.apply(params_logvisc_res, lambda_r*np.ones_like(rho), rho, T))
visc_dilute = viscosity_mie6_dilute(lambda_r, T)
visc_res_result = visc_res_result * visc_dilute

# visc entropy model
visc_entropy_result = visc_entropy_model.apply(params_visc_entropy, alpha, Sres)
visc_entropy_result = viscosity_scaling(rho, T, visc_entropy_result, unscale=True)

print('Shear viscosity model: ', visc_result)
print('Shear viscosity residual model: ', visc_res_result)
print('Viscosity entropy model: ', visc_entropy_result)

Shear viscosity model:  [0.24435199 0.29175086 0.36290351 0.46773517 0.62393206]
Shear viscosity residual model:  [0.2479837  0.29571055 0.36418936 0.46838631 0.62687355]
Viscosity entropy model:  [0.25300686 0.28888959 0.35182701 0.46427662 0.62347342]


In [9]:
##############################
# Thermal conductivity model #
##############################

folder_tcond = 'tcond_models'
hidden_layers = 3
neurons = 30

# Residual logtcond model
prefix = 'logtcond-rho-Tinv-residual-penalty'
seed = 42
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_tcond, target=None, prefix=params_prefix)
params_logtcond_res = {'params': state_restored['params']}
logtcond_res_model = TransportModelResidual_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# logtcond model
prefix = 'logtcond-rho-Tinv-penalty'
seed = 1
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_tcond, target=None, prefix=params_prefix)
params_logtcond = {'params': state_restored['params']}
logtcond_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

# tcond residual entropy model
prefix = 'tcond-entropy-penalty'
seed = 1337
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=folder_tcond, target=None, prefix=params_prefix)
params_tcond_entropy = {'params': state_restored['params']}
tcond_entropy_model = TransportModel_entropy(features=features, output_activation=activation_dicts[activation])

################################
# Testing the different models #
################################

lambda_r = 12
lambda_a = 6 
alpha_vdw = helper_get_alpha(lambda_r, lambda_a)

# defining the state points
rho = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
T = 2. * np.ones_like(rho)
alpha = alpha_vdw * np.ones_like(rho)
Sres = fun_dic_feanneos['entropy_residual_fun'](alpha, rho, T)

# tcond model
tcond_result = np.exp(logtcond_model.apply(params_logtcond, alpha, rho, T))

# tcond residual model
tcond_res_result = np.exp(logtcond_res_model.apply(params_logtcond_res, lambda_r*np.ones_like(rho), rho, T))
tcond_dilute = thermal_conductivity_mie6_dilute(lambda_r, T)
tcond_res_result = tcond_res_result * tcond_dilute

# tcond entropy model
tcond_entropy_result = tcond_entropy_model.apply(params_tcond_entropy, alpha, Sres)
tcond_entropy_result = thermal_conductivity_scaling(rho, T, tcond_entropy_result, unscale=True)

print('Thermal conductivity model: ', tcond_result)
print('Thermal conductivity residual model: ', tcond_res_result)
print('Thermal conductivity entropy model: ', tcond_entropy_result)
      

Thermal conductivity model:  [1.05011208 1.3347346  1.64363532 2.10548117 2.80570599]
Thermal conductivity residual model:  [1.05404151 1.37304217 1.6808723  2.15938115 2.83884181]
Thermal conductivity entropy model:  [1.04035306 1.41270017 1.80823097 2.25433259 2.85097502]
